# LangChain

## Demo code to use LangChain
A quick demo code to use chains in LangChain.

Please ensure you have *.env* file in your HOME/Documents/src/openai/ folder

## Initialize the model first

In [ ]:
!pip install grandalf

In [ ]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from pprint import pprint
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

model = AzureChatOpenAI(deployment_name="gpt-35-turbo")

In [ ]:
tools = [
    {"type": "function", 
         "function": {
            "name": "joke", 
            "description": "Generate a joke",
            "parameters": {
                "type":"object",
                "properties": {
                    "setup": {
                        "type": "string", 
                        "description": "the setup for the joke"
                    },
                    "punchline": {
                        "type": "string", 
                        "description": "the punchline for the joke"
                    }
                },
                "required": ["setup", "punchline"]
            }
        }
    }
]

## Simple LCEL

1. ChatOpenAI to invoke prompt directly
2. Simplest LCEL
3. Inject simple command into ChatOpenAI
4. Simple LCEL with standard format: prompt | model | output_parser
5. Inject tools call into ChatOpenAI
6. Tools call with ToolsParser

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

prompt = ChatPromptTemplate.from_template('tell me a joke about {topic}')

response = model.invoke(prompt.format(topic='programmers'))
print("1. Direct model output:")
pprint(response)
print("\n")

chain_origin = prompt | model
response = chain_origin.invoke({'topic':'programmers'})
print("2. Simple chain output:")
pprint(response)
print("\n")

chain_with_stop_bind = prompt | model.bind(stop=['?'])
response = chain_with_stop_bind.invoke({'topic':'programmers'})
print("3. Chain with stop bind output:")
pprint(response)
print("\n")

chain_with_parser = prompt | model | StrOutputParser()
response = chain_with_parser.invoke({'topic':'programmers'})
print("4. Chain with String Output Parser output:")
pprint(response)
print("\n")

chain_with_tools_bind = prompt | model.bind(tools=tools, tool_choice={'type': 'function', 'function': {'name': 'joke'}})
response = chain_with_tools_bind.invoke({'topic':'programmers'}, config={})
print("5. Chain with Tools Call output:")
pprint(response)
print("\n")

chain_with_tools_bind = prompt | model.bind(tools=tools) | JsonOutputToolsParser()
response = chain_with_tools_bind.invoke({'topic':'programmers'}, config={})
print("6. Chain with Tools Parser output:")
pprint(response)
print("\n")

## Simple Chain

1. Test 1st prompt
2. Test 2nd prompt with hard code input
3. Combine 2 prompts together
4. SimpleSequentialChain

In [ ]:
from operator import itemgetter
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain

prompt1 = ChatPromptTemplate.from_template("what is the year {person} won the gold medal in Olmypics?")
prompt2 = ChatPromptTemplate.from_template("who's the chairman of China in the year of {year}? respond in {language}")

print("1. Chain1 output:")
chain1 = prompt1 | model | StrOutputParser()
response = chain1.invoke({"person": "刘翔"})
pprint(response)
print("\n")

print("2. Chain2 output:")
chain2 = (
    {"year": itemgetter("year"), "language": itemgetter("language")}
     | prompt2 | model | StrOutputParser()
)
response = chain2.invoke({"year": 2004, "language": "Chinese"})
pprint(response)
print("\n")

print("3. LCEL Chain output:")
chain3 = (
    {"year": chain1, "language": itemgetter("language")}
     | prompt2 | model | StrOutputParser()
)
response = chain3.invoke({"person": "刘翔", "language": "Chinese"})
pprint(response)
print("\n")

print("4. SequentialChain output:")
subChain1 = LLMChain(llm=model, prompt=prompt1, output_key="year")
subChain2 = LLMChain(llm=model, prompt=prompt2, output_key="chairman")
chain4 = SequentialChain(chains=[subChain1, subChain2], input_variables=["person", "language"], verbose=False)
response = chain4.invoke({"person": "刘翔", "language": "Chinese"})
pprint(response['chairman'])
print("\n")

print("5. SimpleSequentialChain output:")
prompt2 = ChatPromptTemplate.from_template("who's the chairman of China in the year of {year}?")
subChain1 = LLMChain(llm=model, prompt=prompt1)
subChain2 = LLMChain(llm=model, prompt=prompt2)
chain5 = SimpleSequentialChain(chains=[subChain1, subChain2], verbose=False)
response = chain5.invoke("刘翔")
pprint(response['output'])
print("\n")

## A More Complex Chain

1. input: prompt; output: year
2. input: year; output: USA president's name 1
3. input: year; output: USA president's name 2
4. input: USA president's name 1, USA president's name 2; output: who's popular

<img src="images/langchain-chains.jpg" width="600px">

In [ ]:
from langchain_core.runnables.base import RunnableMap
from langchain.callbacks.tracers import ConsoleCallbackHandler

prompt1 = ChatPromptTemplate.from_template("what's year when 刘翔 won the Olympics gold medal? Please just give the year.")
prompt2 = ChatPromptTemplate.from_template("Who held the position of president of the United States 5 years prior to {year}? Please just give the name.")
prompt3 = ChatPromptTemplate.from_template("Who was the president of the United States 5 years following {year}? Please just give the name.")
prompt4 = ChatPromptTemplate.from_template("who's more popular in USA? {president1} or {president2}?")

print("1. Chain1 output:")
chain1 = prompt1 | model | StrOutputParser()
response = chain1.invoke({})
pprint(response)
print("\n")

print("2. Chain2 output:")
chain2 = {"year":itemgetter("year")} | {
    "president1": prompt2 | model | StrOutputParser(),
    "president2": prompt3 | model | StrOutputParser()
} | prompt4 | model | StrOutputParser()
#response = chain2.invoke({"year": 2004}, config={'callbacks': [ConsoleCallbackHandler()]})
response = chain2.invoke({"year": 2004})
pprint(response)
print("\n")

print("3. Composition Chain output:")
chain_composition = RunnableMap({"year":chain1}) | {
    "president1": prompt2 | model | StrOutputParser(),
    "president2": prompt3 | model | StrOutputParser()
} | prompt4 | model | StrOutputParser()
#response = chain_composition.invoke({}, config={'callbacks': [ConsoleCallbackHandler()]})
response = chain_composition.invoke({})
pprint(response)
chain_composition.get_graph().print_ascii()
print("\n")

print("4. SequentialChain output:")
subChain1 = LLMChain(llm=model, prompt=prompt1, output_key="year", verbose=False)
subChain2 = LLMChain(llm=model, prompt=prompt2, output_key="president1", verbose=False)
subChain3 = LLMChain(llm=model, prompt=prompt3, output_key="president2", verbose=False)
subChain4 = LLMChain(llm=model, prompt=prompt4, output_key="president", verbose=False)
chain_sequence = SequentialChain(chains=[subChain1, 
                                         SequentialChain(
                                             chains=[subChain2, subChain3], 
                                             input_variables=["year"], 
                                             output_variables=["president1", "president2"], 
                                             verbose=False
                                            ), 
                                         subChain4], 
                                 input_variables=[], 
                                 verbose=False)
response = chain_sequence.invoke({})
pprint(response['president'])
print("\n")

## Chain with Json Formatted Output

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from typing import List, Dict, Optional
from enum import Enum

class SortEnum(str, Enum):
    data = 'data'
    price = 'price'

class OrderingEnum(str, Enum):
    ascend = 'ascend'
    descend = 'descend'

class Semantics(BaseModel):
    name: Optional[str] = Field(description="Mobile Package Name",default=None)
    price_lower: Optional[int] = Field(description="Price Floor",default=None)
    price_upper: Optional[int] = Field(description="Price Cap",default=None)
    data_lower: Optional[int] = Field(description="Data Floor",default=None)
    data_upper: Optional[int] = Field(description="Data Cap",default=None)
    sort_by: Optional[SortEnum] = Field(description="Order by Price or Data",default=None)
    ordering: Optional[OrderingEnum] = Field(description="Ascend Order or Descend Order",default=None)

parser = PydanticOutputParser(pydantic_object=Semantics)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Parse user's input as JSON. The format as below：\n{format_instructions}\nDon't output anything which is not mentioned by user",
        ),
        ("user", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain = (
    {"query": RunnablePassthrough()} | prompt | model | parser
)

pprint(chain.invoke("不超过100元且不小于30元，且流量要超过50M的中国电信套餐有哪些"))

## Chain with a VectorStore for Q&A

In [ ]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub

vectorstore = Chroma.from_texts(
    [
        "Sam Altman is the CEO of OpenAI", 
        "Christian Klein is the CEO of SAP",
        "PHP is the best programming language",
        "Virtual Threads in recent Java versions send WebFlux back home"
    ], embedding=AzureOpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

qachain_legacy = RetrievalQA.from_chain_type(model, retriever=retriever)

qachain_lcel = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt
    | model
    | StrOutputParser()
)

qachain_lcel2 = (
    {"question": RunnablePassthrough(), "context": RunnablePassthrough() | retriever} 
    | prompt 
    | model 
    | StrOutputParser()
)


combine_docs_chain = create_stuff_documents_chain(model, retrieval_qa_chat_prompt)
qachain_neo = create_retrieval_chain(retriever, combine_docs_chain)

question = "SAP的CEO是谁"

print("1. RetrievalQA chain output:")
pprint(qachain_legacy.invoke(question)['result'])
print("\n")

print("2. LCEL chain output:")
pprint(qachain_lcel.invoke(question))
print("\n")

print("3. LCEL2 chain output:")
pprint(qachain_lcel2.invoke(question))
print("\n")

print("4. create_retrieval_chain chain output:")
pprint(qachain_neo.invoke({"input":question})['answer'])
print("\n")

## Router Chain Example

In [ ]:
from langchain.chains import create_tagging_chain_pydantic  
from langchain.schema.runnable import RouterRunnable, RunnableLambda, RunnableBranch
from langchain_core.prompts import PromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
  
class PromptToUse(BaseModel):  
    """Used to determine which prompt to use to answer the user's input."""  
      
    name: str = Field(description="Should be one of `math`, `chemistry`, `physics`")  

tagger = create_tagging_chain_pydantic(PromptToUse, model)

math_template = "You are a math genius. Answer the question: {input}"
physics_template = "You are a physics professor. Answer the question: {input}"
chemistry_template = "You are a chemistry expert. Answer the question: {input}"

chain1 = ChatPromptTemplate.from_template(math_template) | model
chain2 = ChatPromptTemplate.from_template(physics_template) | model
chain3 = ChatPromptTemplate.from_template(chemistry_template) | model

#router runnable part
router_runnable = RouterRunnable({"math": chain1, "physics": chain2, "chemistry": chain3})

chain_runnable = {
"key": {"input": lambda x: x["input"]} | tagger | (lambda x: x['text'].name),
"input": {"input": lambda x: x["input"]}
} | router_runnable

#runnable lambda part
def routeFunc(info):
    if "math" in info["topic"].lower():
        return chain1
    elif "physics" in info["topic"].lower():
        return chain2
    else:
        return chain3

chain_judge = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `math`, `chemistry`, or `physics`.

Do not respond with more than one word.

<input>
{input}
</input>

Classification:"""
    )
    | model
    | StrOutputParser()
)

chain_lambda = {
    "topic": chain_judge, "input": lambda x: x["input"]
} | RunnableLambda(routeFunc)

#runnable branch part
branch = RunnableBranch(
    (lambda x: "math" in x["topic"].lower(), chain1),
    (lambda x: "physics" in x["topic"].lower(), chain2),
    (lambda x: "chemistry" in x["topic"].lower(), chain3),
    chain3
)
chain_branch = {"topic": chain_judge, "input": lambda x: x["input"]} | branch

#legacy router chain part
prompt_infos = [
    {
        "name": "math",
        "description": "answer math puzzles",
        "prompt_template": math_template,
    },
    {
        "name": "physics",
        "description": "answer physics puzzles",
        "prompt_template": physics_template,
    },
    {
        "name": "chemistry",
        "description": "answer chemstry puzzles",
        "prompt_template": chemistry_template,
    },
]

destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]  
    prompt_template = p_info["prompt_template"] 
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=model, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(model, router_prompt)

chain_multiprompt = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=LLMChain(llm=model, prompt=ChatPromptTemplate.from_template(chemistry_template)),
    verbose=True,
)


my_question = {"input": "explain redox reaction?"}
print("1. RouterRunnable output:")
pprint(chain_runnable.invoke(my_question))
chain_runnable.get_graph().print_ascii()
print('\n')

print("2. RunnableLambda output:")
pprint(chain_judge.invoke(my_question))
pprint(chain_lambda.invoke(my_question))
chain_lambda.get_graph().print_ascii()
print('\n')

print("3. RunnableBranch output:")
pprint(chain_branch.invoke(my_question))
chain_branch.get_graph().print_ascii()
print('\n')

print("4. LLMRouterChain output:")
pprint(chain_multiprompt.invoke(my_question))
chain_multiprompt.get_graph().print_ascii()
print('\n')